## TRAPI query results

In [105]:
import requests
import json

search_string = 'Buprenorphine'
results = requests.post(f'https://name-resolution-sri.renci.org/lookup?string={search_string}&offset=0&limit=10')
results_json = results.json()
input_node_id_list = list(results_json.keys())

nn_query = {
  "curies": input_node_id_list,
  "conflate": True
}

results = requests.post('https://nodenormalization-sri.renci.org/get_normalized_nodes',json=nn_query)

query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": input_node_id_list,
            "categories": ["biolink:ChemicalEntity"]
          },
          "n01": {
              "categories": ["biolink:BiologicalProcessOrActivity","biolink:Gene","biolink:Pathway"]
          },
          "n02": {
            "ids": ["HP:0001337"],
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }

robokop_submit_url = "https://automat.renci.org/robokopkg/1.3/query"
response = requests.post(robokop_submit_url,json=query)
print(response.status_code)
number_pathway_results = len(response.json()['message']['results'])
print(len(response.json()['message']['results']))

200
8


In [28]:
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
write_dir = Path("output/TRAPI",str(dt_string))
write_dir.mkdir(parents=True, exist_ok=True)

In [106]:
import pandas as pd
from pathlib import Path

nodes = response.json()['message']['results'][0]['node_bindings'].keys()
output_dict = {}
for key in nodes:
    output_dict[key] = {}
    
i = 0
for entry in response.json()['message']['results']:
    # for edge_id in edges:
    #     output_dict[edge_id][i] = entry['edge_bindings'][edge_id][0]['id']
    for node_id in nodes:
        output_dict[node_id][i] = entry['node_bindings'][node_id][0]['id']
    i = i + 1
    
node_list = [x for x in output_dict.keys() if 'n' in x]

for node_id in node_list:
    for node_index, node_value in output_dict[node_id].items():
        # Getting the 'name' label using the node ID from the knowledge graph
        output_dict[node_id][node_index] = response.json()['message']['knowledge_graph']['nodes'][node_value]['name']
        
pp.pprint(output_dict)
json_str = json.dumps(output_dict, indent=4)
df = pd.read_json(json_str)
df.to_csv(Path(write_dir,'pathways_TRAPI_automat.csv'))

{    'n00': {    0: 'Buprenorphine',
                 1: 'Buprenorphine',
                 2: 'Buprenorphine',
                 3: 'Buprenorphine',
                 4: 'Buprenorphine',
                 5: 'Brixadi',
                 6: 'Buprenorphine',
                 7: 'Buprenorphine'},
     'n01': {    0: 'CYP2D6',
                 1: 'CYP2D6',
                 2: 'CYP2D6',
                 3: 'CYP2D6',
                 4: 'OPRM1',
                 5: 'OPRM1',
                 6: 'CYP2D6',
                 7: 'CYP2D6'},
     'n02': {    0: 'Limb tremor',
                 1: 'Pill-rolling tremor',
                 2: 'Tremor',
                 3: 'Postural tremor',
                 4: 'Asterixis',
                 5: 'Asterixis',
                 6: 'Resting tremor',
                 7: 'Action tremor'}}


In [107]:
colnames = df.columns.values.tolist()
colnames.sort()

combined_node_list = ["_".join([row[0].replace(" ", "_"), row[1].replace(" ", "_"), row[2].replace(" ", "_")]) for row in df[colnames].to_numpy()]
# pp.pprint(combined_node_list)
for node_set in combined_node_list:
    print(os.path.join(write_dir,node_set+".txt"))

output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Limb_tremor.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Pill-rolling_tremor.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Tremor.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Postural_tremor.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_OPRM1_Asterixis.txt
output\Cypher_automat\2023-03-29_081046\Brixadi_OPRM1_Asterixis.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Resting_tremor.txt
output\Cypher_automat\2023-03-29_081046\Buprenorphine_CYP2D6_Action_tremor.txt


In [108]:
from collections import Counter
import json
import pprint
pp = pprint.PrettyPrinter(indent=5)

for i in range(number_pathway_results):
    # if i >= 1:
        # break
    print(f"Pathway result: {combined_node_list[i]}")
    edge_bindings = response.json()['message']['results'][i]['edge_bindings']
    # print(edge_bindings)

    edge_ids = []
    for edge_name, edge_list in edge_bindings.items():
        edge_ids.append({edge_name: [x['id'] for x in edge_list]})

    # print(edge_ids)
    string_out_list = []
    for edge_dict in edge_ids:
        for edge_name, edge_list in edge_dict.items():
            # print(f"{edge_name}: {edge_list}")
            for edge_id in edge_list:
                subject_id = response.json()['message']['knowledge_graph']['edges'][edge_id]['subject']
                subject = response.json()['message']['knowledge_graph']['nodes'][subject_id]['name']
                predicate = response.json()['message']['knowledge_graph']['edges'][edge_id]['predicate']
                object_id = response.json()['message']['knowledge_graph']['edges'][edge_id]['object']
                object = response.json()['message']['knowledge_graph']['nodes'][object_id]['name']
                # print(f"\t{subject} -> {predicate} -> {object}")
                string_out = f"{subject} -> {predicate} -> {object}"
                string_out_list.append(string_out)
                # if edge_name == 'e01':
                #     print(response.json()['message']['knowledge_graph']['edges'][edge_id]['attributes'])
    string_out_dict = dict(Counter(string_out_list).items())
    pp.pprint(string_out_dict)
    print("")
    
    with open(os.path.join(write_dir,combined_node_list[i]+".txt"), 'w') as convert_file:
        convert_file.write(json.dumps(string_out_dict))
        

Pathway result: Buprenorphine_CYP2D6_Limb_tremor
{    'Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'CYP2D6 -> biolink:genetic_association -> Limb tremor': 1}

Pathway result: Buprenorphine_CYP2D6_Pill-rolling_tremor
{    'Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'CYP2D6 -> biolink:genetic_association -> Pill-rolling tremor': 1}

Pathway result: Buprenorphine_CYP2D6_Tremor
{    'Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'CYP2D6 -> biolink:genetic_association -> Tremor': 2}

Pathway result: Buprenorphine_CYP2D6_Postural_tremor
{    'Buprenorphine -> biolink:a

## Cypher query results (automat.renci.org)

In [84]:
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
write_dir = Path("output/Cypher_automat",str(dt_string))
write_dir.mkdir(parents=True, exist_ok=True)

In [110]:
cypher = f"MATCH (n0_0:`biolink:ChemicalEntity`)-[r0_0]-(n1_0:`biolink:Gene`)-[r1_0]-(n2_0:`biolink:DiseaseOrPhenotypicFeature`) WHERE n0_0.name IN ['Buprenorphine'] AND n2_0.name IN ['Tremor'] RETURN [startNode(r0_0),[type(r0_0),properties(r0_0)],endNode(r0_0)] as edge_1, [startNode(r1_0),[type(r1_0),properties(r1_0)],endNode(r1_0)] as edge_2, [n0_0.name, n1_0.name, n2_0.name] as node_names LIMIT 100"

j = {'query': cypher}
results = requests.post('https://automat.renci.org/robokopkg/cypher',json=j)
results_json = results.json()
print(results_json['results'][0]['columns'])
column_names = results_json['results'][0]['columns']
print(results.status_code)

['edge_1', 'edge_2', 'node_names']
200


In [119]:
i = 0

string_out_list = []
for result in results_json['results'][0]['data']:
    # print(f'Result: {i+1}')
    j = 0
    for item in result['row'][0:2]:
        # print(item)
        # print(f"{column_names[j]}: {item[0]['name']} -> {item[1][0]} -> {item[2]['name']}")
        # print(f"Edge properties: {item[1][1]}\n")
        # print(f"{item[0]['name']} -> {item[1][0]} -> {item[2]['name']}")
        # string_out = f"{item[0]['name']} -> {item[1][0]} -> {item[2]['name']}"
        string_out_test = f"{item[0]['name']} -> {item[1][0]} -> {item[2]['name']}||{item[1][1]}"
        if string_out_test not in string_out_list:
            string_out_list.append(string_out_test)
        j = j + 1
    # print("")

    i = i + 1
    # if i > 0:
    #     break
    
combined_node_list = "_".join(results_json['results'][0]['data'][0]['row'][2]).replace(" ", "_")
print(combined_node_list)

string_out_list = [i.split('||', 1)[0] for i in string_out_list]

string_out_dict = dict(Counter(string_out_list).items())
pp.pprint(string_out_dict)

with open(os.path.join(write_dir,combined_node_list+".txt"), 'w') as convert_file:
    convert_file.write(json.dumps(string_out_dict))

Buprenorphine_CYP2D6_Tremor
{    'Buprenorphine -> biolink:affects -> CYP2D6': 1,
     'Buprenorphine -> biolink:directly_physically_interacts_with -> CYP2D6': 1,
     'CYP2D6 -> biolink:affects -> Buprenorphine': 1,
     'CYP2D6 -> biolink:genetic_association -> Tremor': 2}
